# Data Literacy
#### University of Tübingen, Winter Term 2020/21
## Exercise Sheet 3
&copy; 2020 Prof. Dr. Philipp Hennig & Jonathan Wenger

This sheet is **due on Tuesday 24 November 2020 at 12noon sharp (i.e. before the start of the lecture).**

---

## Experimental Design

This week we will take another look at experimental design, in particular you will come up with a testing strategy for COVID-19 that tests the entire population of a state with a limited testing budget by pooling samples from the population. This can also be interpreted as a type of resource allocation problem. We would like to use the limited amount of tests that we have to test as many people as possible in a timely fashion.

### RT-PCR Testing for COVID-19

One of the main tests used for COVID-19 diagnostic is the RT-PCR test which quantifies the amount of viral RNA, typically from a nasal or throat swab. In a nutshell the genetic information of the virus is reversely transcribed (RT) into DNA which is then multiplied via a chain reaction (PCR) to make it detectable. 

<img src="https://idtxs3.imgix.net/si/40000/64/84.png?w=1200&fit=fill&bg=ffffff&border=0&q=50" alt="Reverse Transcriptase - Polymerase Chain Reaction" style="width: 600px;"/>

One feature of this type of test is that one can pool multiple swabs into a single sample which then gets analyzed. In this exercise we will aim to exploit this fact to design a near-optimal testing strategy.

In [ ]:
# Make inline plots vector graphics
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("pdf", "svg")

# Plotting setup
import matplotlib
import matplotlib.pyplot as plt

# Package imports
import numpy as np
import pandas as pd

## Pooled Testing

A pooled testing strategy works as follows. For each bucket of pooled samples determine whether viral RNA is present. If not, all samples in that bucket are deemed negative. If the result comes up positive, create two new pooled buckets out of the samples from the original bucket. You can now iterate this strategy for each bucket until the entire population has received a test result.

<img src="https://members.loria.fr/ADeleforge/wp-content/blogs.dir/192/files/sites/192/2020/04/pool-testing1.png" alt="Test Pooling" style="width: 600px;"/>


**Task:** Assuming a population of size $n$ and even splits, how many *rounds of tests* do you need in the best and worst case? How often would you split a swab sample from an individual from Baden-Württemberg (population: 11 million) to make sure they only have to show up for a test once?

### Synthetic Test Data from Baden Württemberg

In preparation for the implementation and evaluation of a pooled testing strategy, we will generate a synthetic dataset representing one million people from Baden-Württemberg. The geographic and demographic information, as well as the infection statistics are taken from the RKI and the german census. You can assume in the following that each of the individuals considered has a unique ID from $0-999999$.

*Disclaimer:* This population has been synthetically generated and does _not_ reflect real patient data. Do _not_ draw conclusions about COVID-19 from this data!

In [ ]:
# Sample population of Baden-Württemberg
n_population_bw = 10 ** 6
population_ids = np.arange(0, n_population_bw)

In [ ]:
%run data/sample_test_results

# Create a synthetic test dataset of test results for Baden-Württemberg
sample_test_data_bw_ground_truth = sample_test_results(n_population_bw)
sample_test_data_bw = sample_test_data_bw_ground_truth.drop(
    columns=["id_county", "population", "total_population_county", "positive_test"]
)

# Testing function describing result of PCR test for a pooled sample
positive_test_data_bw = positive_test_data_bw = sample_test_data_bw_ground_truth[
    sample_test_data_bw_ground_truth.positive_test == 1
]
positive_test_data_bw_bool = np.zeros(n_population_bw, dtype=bool)
positive_test_data_bw_bool[positive_test_data_bw.index] = True


def is_positive_batch(batch_indices):
    return np.any(positive_test_data_bw_bool[batch_indices])

**Task:** Use the `is_positive_batch` function to check a few custom batches of samples of your choosing.

### Naive Pooling

We will now begin by implementing a naive pooling strategy. Assume you can pool up to $10,000$ swabs from patients per PCR test (also called batch). After initial assignment, we test each batch and then split it (randomly) in half. Repeat until every batch is either tested negative or has size one.

**Task:** Implement a splitting function which randomly splits a batch into $n$ splits of roughly equal size.

*Hint:* For all of the following exercises it is a good idea to search through the NumPy documentation for helpful array manipulation routines.

In [ ]:
def batch_split(batch, n_splits):
    """
    Random batch splitting strategy.
    
    Parameters
    ----------
    batch : array-like
        IDs of samples to be split.
    n_splits : int
        Number of splits to generate.
    
    Returns
    -------
    batches : list
        List of new batches after splitting.
    """
    raise NotImplementedError

**Task:** Implement a test and split function which takes a batch and the splitting method from above. It then tests the current batch and either returns the test result if the batch is of size one or the test comes up negative or recursively traverses down the tree by splitting the batch into two new batches. Make sure your function also returns statistics on the number of tests performed so far and the number of testing rounds (or tree depth of the corresponding binary tree).

In [ ]:
def test_and_split(batch, splitting_method=batch_split):
    """
    Test the current batch and recursively returns the test results for all samples in the batch.
    
    Tests the current batch and then recursively splits the batch into smaller batches until all
    samples making up the batch are tested.
    
    Parameters
    ----------
    batch : array-like
        Batch described by integer IDs.
    splitting_method : callable
        Callable returning a set of batches given a single batch.
        
    Returns
    -------
    test_results : array-like
        Array with one entry for each ID in batch describing a positive or negative test result.
    n_tests : int
        Number of tests performed for this batch.
    n_rounds : int
        Number of rounds of tests performed for this batch. Equals the maximum number of tests 
        performed on a single swab sample.
    """
    # Test batch
    is_positive = is_positive_batch(batch)
    
    # Statistics
    n_tests = 1
    n_rounds = 1

    if is_positive:
        if len(batch) > 1:
            # Split batch and recurse down tree
            new_batches = None # TODO
            test_results_left, n_tests_left, n_rounds_left = None # TODO
            test_results_right, n_tests_right, n_rounds_right = None # TODO

            # Collate results
            test_results = None # TODO
            n_tests = None # TODO
            n_rounds = None # TODO
        else:
            # Single positive test
            test_results = None # TODO
    else:
        if len(batch) == 1:
            # Single negative test
            test_results = None # TODO
        else:
            # Negative batch
            test_results = None # TODO

    return test_results, n_tests, n_rounds

**Task:** Implement the complete pooled testing strategy to test the given population sample of Baden-Württemberg.

In [ ]:
def pooled_testing(population_ids, max_batch_size=10000, splitting_method=batch_split):
    """
    Test a set of swab samples for COVID-19 with a pool-and-split strategy.

    Parameters
    ----------
    population_ids : array-like, shape=(n,)
        IDs of population to be tested.
    max_batch_size : int
        Maximum size of a pooled/batched sample.
    splitting_method : callable
        Callable taking a batch as input and splitting it into a given 
        number of batches.

    Returns
    -------
    test_results : array-like, shape=(n,)
        Boolean array returning whether the associated test is positive.
        Assumed to be ordered as ``population_ids``.
    n_tests : int
        Number of tests performed.
    n_rounds : int
        Maximum number of tests performed on a single swab sample.
    """
    # Distribute samples into batches
    n_splits = None # TODO
    test_batches = None # TODO

    # Test each batch with splitting strategy
    test_results = np.array([])
    n_tests = []
    n_rounds = []
    
    for batch in test_batches:    
        # TODO

    return test_results, n_tests, n_rounds

**Task:** How many tests does the pooled testing strategy require for the test population?

*Note:* This computation may take a few seconds on your machine. Experiment with smaller populations first to be sure your algorithm is correct. The time taken likely does not reflect poor algorithm design on your end, but rather the lookup whether a test is positive or not.

In [ ]:
%%time
# Run pooled testing strategy
np.random.seed(42)
# TODO

## Testing Strategy based on Traits

If we have collected data in the past about traits that are predictive of a COVID-19 infection, we can make use of this information in our testing strategy. Ideally, we would like to achieve a 50% probability of obtaining a positive result from a pooled sample. In this way, in expectation we obtain a result for half of the remaining population in each round. However, this is not necessarily the case given random batch assignments as we have done previously.

### Estimating the Probability of a Positive Test

**Task:** Load the available testing data from the RKI from Baden-Württemberg in November and corresponding population statistics from the German census. Compute some appropriate summary statistics with regards to `gender`, `name_county` and `age_group` and try to get a feel for which population groups are particularly susceptible.

In [ ]:
# Load RKI data from November
data_rki_bw = pd.read_csv("data/data_rki_bw.csv")
#display(data_rki_bw.head())

data_census_bw = pd.read_csv("data/census_2011_bw.csv")
#display(data_census_bw.head())

# Merge dataframes

# Summary statistics


**Task:** Describe in words a model for the probability of an individual having a positive test based on the available traits and summary statistics from above. 

*Note:* If you've taken other ML courses before, you can likely come up with a quite sophisticated model. Feel free to train your own classifier and see how well you can do!

For brevity we provide estimated probabilities of obtaining a positive test for the test data we generated earlier. These estimates are computed as empirical frequencies from the RKI case data in November.

In [ ]:
# Probabilities for tested individuals to be positive based on empirical data
test_positive_probs = sample_test_data_bw_ground_truth.prob_positive_test

### Sample Allocation

We will now try to make use of this additional information we have on the tests. 

**Task:** Given a set of probabilities of obtaining a positive test, write a function which greedily allocates the associated tests into buckets of pooled tests such that the probability of obtaining at least one positive test in each bucket is close to 50%.

In [ ]:
# Greedy allocation algorithm based on probability of a positive test
def batch_split_greedy(batch, n_splits):
    """
    Split a batch of samples into smaller batches based on probabilities.
    
    The given batch of swab samples is split into ``n_splits`` based on 
    probabilities for each sample to come up positive. The new batches are
    formed in a greedy fashion.
    
    Parameters
    ----------
    batch : array-like
        IDs of samples to be split.
    n_splits : int
        Number of splits to generate.
    
    Returns
    -------
    batches : list
        List of new batches after splitting.
    """
    # Initialization
    new_batches = []

    # Probability of positive tests
    p = test_positive_probs[batch]
    
    # Shuffle samples
    perm = np.random.permutation(len(batch))
    batch = batch[perm]
    p = p[perm]

    # Iterate through batch and compute running probability of positive test
    for i, p_current_test in enumerate(p):

        # Compute probability of all tests in batch being negative
        prob_all_negative =  # TODO

        # Form a batch if probability of positive test exceeds 50%
        # TODO

    return new_batches

### Refined Testing Strategy

**Task:** Evaluate your testing strategy on the provided synthetic test data set. How many tests did your approach need? What was the maximal number of tests per batch? Did this strategy perform better than the naive pooled strategy?

*Note:* As above this may take a few moments depending on your computing setup.

In [ ]:
%%time
# Run pooled testing strategy
np.random.seed(42)
# TODO